In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import os
from PIL import Image
import numpy as np
import shutil
from sklearn.model_selection import train_test_split


In [ ]:
# Path to your current dataset directory that contains all the class folders
source_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), "model/data/olympic\ dataset/JO") # Change this path if needed
base_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), "model/data")  # Base path for new image structure

train_path = os.path.join(base_path, 'train')
val_path = os.path.join(base_path, 'val')

# Desired image size (width, height)
desired_size = (256, 256)

# Create train and val directories for each class
if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(val_path):
    os.makedirs(val_path)

classes = os.listdir(source_path)

for class_name in classes:
    class_dir = os.path.join(source_path, class_name)
    if not os.path.isdir(class_dir):
        continue

    # Create class subdirectories in train and val
    os.makedirs(os.path.join(train_path, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_path, class_name), exist_ok=True)

    # Get all images for this class
    images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]

    # Split images into train (80%) and val (20%)
    train_images, val_images = train_test_split(images, test_size=0.2, random_state=42)

    # Function to resize and save image
    def resize_and_save_image(src_path, dest_path, size):
        try:
            with Image.open(src_path) as img:
                img_resized = img.resize(size, Image.LANCZOS)
                img_resized.save(dest_path)
        except Exception as e:
            print(f"Error resizing image {src_path}: {e}")

    # Move and resize images to train directory
    for image in train_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(train_path, class_name, image)
        resize_and_save_image(src, dst, desired_size)

    # Move and resize images to val directory
    for image in val_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(val_path, class_name, image)
        resize_and_save_image(src, dst, desired_size)

print("Dataset has been organized and resized successfully.")

Dataset has been organized and resized successfully.


In [4]:
from ultralytics import YOLO
model = YOLO("yolo11n-cls.pt")

100%|██████████| 5.52M/5.52M [00:00<00:00, 51.5MB/s]


In [5]:
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.5.1+cu118 (CPU)


In [ ]:
# Train the model
# Train the model
train_results = model.train(
    data = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), "model/data"),  # path to dataset YAML
    epochs=50,  # number of training epochs
    imgsz=256,  # training image size
    device="cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

Ultralytics 8.3.27  Python-3.11.9 torch-2.5.1+cu118 CPU (AMD Ryzen 7 7735HS with Radeon Graphics)
engine\trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=C:\Users\as032\iCloudDrive\Documents\UTD\Fall 24\Multimedia Systems\Project\model\data, epochs=50, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fals

train: Scanning C:\Users\as032\iCloudDrive\Documents\UTD\Fall 24\Multimedia Systems\Project\model\data\train... 4588 images, 0 corrupt: 100%|██████████| 4588/4588 [00:01<00:00, 2565.62it/s]


train: New cache created: C:\Users\as032\iCloudDrive\Documents\UTD\Fall 24\Multimedia Systems\Project\model\data\train.cache


val: Scanning C:\Users\as032\iCloudDrive\Documents\UTD\Fall 24\Multimedia Systems\Project\model\data\val... 1184 images, 0 corrupt: 100%|██████████| 1184/1184 [00:00<00:00, 2476.22it/s]


val: New cache created: C:\Users\as032\iCloudDrive\Documents\UTD\Fall 24\Multimedia Systems\Project\model\data\val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 256 train, 256 val
Using 0 dataloader workers
Logging results to runs\classify\train
Starting training for 50 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

      50/50         0G     0.0512         12        256: 100%|██████████| 287/287 [01:24<00:00,  3.38it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 37/37 [00:07<00:00,  5.03it/s]

                   all      0.846       0.97



50 epochs completed in 1.344 hours.
Optimizer stripped from runs\classify\train\weights\last.pt, 3.3MB
Optimizer stripped from runs\classify\train\weights\best.pt, 3.3MB

Validating runs\classify\train\weights\best.pt...
Ultralytics 8.3.27  Python-3.11.9 torch-2.5.1+cu118 CPU (AMD Ryzen 7 7735HS with Radeon Graphics)
YOLO11n-cls summary (fused): 112 layers, 1,606,727 parameters, 0 gradients, 3.2 GFLOPs
train: C:\Users\as032\iCloudDrive\Documents\UTD\Fall 24\Multimedia Systems\Project\model\data\train... found 4588 images in 63 classes  
val: C:\Users\as032\iCloudDrive\Documents\UTD\Fall 24\Multimedia Systems\Project\model\data\val... found 1184 images in 63 classes  
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 37/37 [00:06<00:00,  5.30it/s]


                   all      0.853      0.967
Speed: 0.0ms preprocess, 3.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train
